<h4>-----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.</h4>
<h5> THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN <br> THE  SOFTWARE.<br>-----------------------------------------------------------------------------</h5>

In [ ]:
from arena_api.system import system
import time

#### Pixel Correction
> This example introduces the basics of pixel correction. A single arbitrary pixel is chosen and added to the device's pixel correction list. These changes are then saved to the camera before being removed.

In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

##### Pixel values
> This example does not search for pixels to correct, but instead just uses arbitrary pixels to demonstrate what would be done if a bad pixel were found.

In [ ]:
PIXEL_X = 333
PIXEL_Y = 444

In [ ]:
nodemap = device.nodemap
nodes = nodemap.get_node(['DefectCorrectionEnable','DefectCorrectionCount',
'DefectCorrectionPositionX','DefectCorrectionPositionY',
'DefectCorrectionGetNewDefect', 'DefectCorrectionIndex', 
'DefectCorrectionApply', 'DefectCorrectionSave', 'DefectCorrectionRemove'])

##### Enable pixel correction

In [ ]:
nodes['DefectCorrectionEnable'] = True

pixel_correction_count = nodes['DefectCorrectionCount'].value

print("Initial pixel correction count: " + str(pixel_correction_count))

##### Add new pixel to end of correction list and get its index
>Getting a new defect automatically updates the pixel correction index.

In [ ]:
nodes['DefectCorrectionGetNewDefect'].execute()

pixel_correction_updated_index = nodes['DefectCorrectionIndex'].value

print("Pixel correction updated index: " + str(pixel_correction_updated_index))

##### Set the position of pixels to be corrected

In [ ]:
nodes['DefectCorrectionPositionX'] = PIXEL_X

nodes['DefectCorrectionPositionY'] = PIXEL_Y

##### Apply correction list
> Once the pixel corrections are applied, they will take effect immediately. However, by power-cycling the camera, the defect list will go back to its default values.

In [ ]:
nodes['DefectCorrectionApply'].execute()
print("Applied correction list")

##### Save correction to camera
>Optionally write the correction to the camera to make the changes persistent (the camera can still be set to default by executing the DefectCorrectionRestoreDefault node)

In [ ]:
nodes['DefectCorrectionSave'].execute()
print("Saved correction to camera")

##### Remove all pixels set through this example
> The index is updated so that there are no empty indices after removal regardless of position

In [ ]:
i = pixel_correction_updated_index
while ( i >= pixel_correction_count):
    nodes['DefectCorrectionIndex'] = i;

    x = nodes['DefectCorrectionPositionX']
    y = nodes['DefectCorrectionPositionY']
    print("Pixel index: " + str(nodes['DefectCorrectionIndex'])
    + " (x: " + str(x) + ", y:" + str(y) + ")")

    if x == PIXEL_X and y == PIXEL_Y:
        print("Pixels match. Remove Pixel")
        nodes['DefectCorrectionRemove'].execute()
        print("Pixel Removed")
    else:
        print("Does not match");
    
    i = i-1

##### Clean up ----------------------------------------------------------------
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
system.destroy_device();